# TreNet Implementation

### Authors: Nathan Ng

### TreNet Implementation

- Feeds trend duration and slope into LSTM 
- Feeds corresponding data points with trends to CNN stack 
- Combines output of LSTM and CNN in feature fusion layer
- Outputs final prediction with fully connected layer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Use GPU if available for PyTorch
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)

## Data Pre-Processing

In [3]:
def pad_data(data):
    """
    Pad all rows with 0's to match longest row 
    """
    max_len = data.apply(len).max()
    
    pad_row = lambda x: ([0] * (max_len - len(x))) + x
    padded_data = data.apply(pad_row)
    return padded_data

In [4]:
def convert_data_points(data):
    """
    Takes in dataframe of [duration, slope, data_points]
    Returns tensor of trend data and tensor of corresponding data points
    """
    
    # Extract trends data 
    trends = data[['duration', 'slope']]
    trends = torch.tensor(trends.values)
    
    # Extract data points 
    data_pts = data['data_points'].apply(ast.literal_eval)
    data_pts = pad_data(data_pts)
    data_pts = torch.tensor(data_pts)
    
    return trends, data_pts

In [5]:
"""
Scales data using Sklearn's MinMaxScaler
Generalized to accept tensors 
"""
class Scaler():
    def __init__(self):
        self.scaler = MinMaxScaler()
        
    def fit_transform(self, data):
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
            
        # Transform data 
        if len(data.shape) == 1: 
            scaled_data = self.scaler.fit_transform(data.reshape(-1, 1)).flatten()
        else: 
            scaled_data = self.scaler.fit_transform(data)
        
        # Return tensor of scaled data
        return torch.tensor(scaled_data, dtype=torch.float)
    
    def inverse_transform(self, data): 
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
        
        inverse_data = self.scaler.inverse_transform(data)
        
        # Return tensor of inverse data
        return torch.tensor(inverse_data, dtype=torch.float)

In [6]:
"""
Extracts m sequential data to use to predict n next data 
"""
def extract_data(data, num_input, num_output):
    num_rows = data.shape[0] - num_input - num_output
    if len(data.shape) == 2: 
        input_data = torch.zeros(num_rows, num_input, data.shape[-1])
        output_data = torch.zeros(num_rows, num_output, data.shape[-1])
    else: 
        input_data = torch.zeros(num_rows, num_input)
        output_data = torch.zeros(num_rows, num_output)
    
    for i in range(num_rows):
        input_data[i] = (data[i:i+num_input])
        output_data[i] = (data[i+num_input:i+num_input+num_output])
    return input_data, output_data

In [7]:
"""
Separates data into train, validation, and test sets
props: (train, valid)
"""
def train_valid_test_split(X, y=None, props=None):
    if not props: 
        props = (0.5, 0.25)
    elif len(props) != 3: 
        print("Wrong number of parameters")
        return None
    
    train_size = int(X.shape[0] * props[0])
    valid_size = int(X.shape[0] * props[1])
    
    X_train = X[:train_size].to(device)
    X_valid = X[train_size: (train_size + valid_size)].to(device)
    X_test = X[(train_size + valid_size):].to(device)
    
    if y != None: 
        y_train = y[:train_size].to(device)
        y_valid = y[train_size: (train_size + valid_size)].to(device)
        y_test = y[(train_size + valid_size):].to(device)
        
        return X_train, y_train, X_valid, y_valid, X_test, y_test
    
    return X_train, X_valid, X_test

## TreNet Model

In [8]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        
        # Initialize hidden dimenision and layers 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.input_dim = input_dim
        
        # Initialize deep learning models
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        
        # Reshape data if needed
        if len(x.shape) != 3: 
            x = x.reshape(x.shape[0], -1, self.input_dim)
        
        # Run data through model
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [9]:
class TreNetCNN(nn.Module):
    def __init__(self, num_data, layers=None, num_filters=None, dropout=None, conv_size=3, pooling_size=3, output_size=2):
        """
        layers (int): Number of cnn stacks to create 
        num_filters (list(int)): Number of CNN filters corresponding to same index stacks
        dropout (list(float)): Probability of dropout corresponing to same index stack
        convsize (int, list(int)): Size of filter sizes 
        """
        super(TreNetCNN, self).__init__()
        self.num_data = num_data
        self.layers = layers
        self.num_filters = num_filters
        self.dropout = dropout
        self.conv_size = conv_size
        self.pooling_size = pooling_size
        self.output_size = output_size
        self.cnn_stack = self.create_cnn_stack()
    
    def create_cnn_stack(self):
        # Initialize default stack settings
        if not self.layers:
            self.layers = 2
        if not self.num_filters:
            self.num_filters = [128] * self.layers
        if not self.dropout:
            self.dropout = [0.0] * self.layers
        if type(self.conv_size) == int:
            self.conv_size = [self.conv_size] * self.layers
        
        # Create cnn stacks 
        cnn_stacks = []
        num_channels = 1
        updated_data = self.num_data
        for i in range(self.layers):
            cnn_stack = nn.Sequential(
                nn.Conv1d(in_channels=num_channels, out_channels=self.num_filters[i], kernel_size=self.conv_size[i]),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=self.pooling_size, stride=1),
                nn.Dropout(p=self.dropout[i])
            )
            num_channels = self.num_filters[i]
            cnn_stacks.append(cnn_stack)
            
            # Keep track of current size of data 
            updated_data = updated_data - self.conv_size[i] + 1 - self.pooling_size + 1
            new_data_size = self.num_filters[i] * updated_data
            
        # Add fully connected layer at end to output trend duration and slope
        output_layer = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(new_data_size, self.output_size)
        )
        cnn_stacks.append(output_layer)
            
        # Combine cnn stacks 
        return nn.Sequential(*cnn_stacks)
    
    def forward(self, x):
        x = torch.reshape(x, (x.shape[0], 1, -1))
        output = self.cnn_stack(x)
        return output

In [10]:
class TreNet(nn.Module):
    def __init__(self, LSTM_params, CNN_params, feature_fusion, output_dim):
        super(TreNet, self).__init__()
        
        # Set number of parameters for feature fusion layer
        LSTM_params['output_dim'] = feature_fusion
        CNN_params['output_size'] = feature_fusion
        
        self.lstm = LSTM(**LSTM_params)
        self.cnn = TreNetCNN(**CNN_params)
        self.fusion = nn.Linear(feature_fusion, output_dim)
        self.cutoff = CNN_params['num_data']
        
    def forward(self, data):
        trends, data = data
        
        # Run trends through LSTM 
        lstm_out = self.lstm(trends)
        
        # Set cutoff for CNN stock prices
        cutoff_data = torch.zeros(data.shape[0], self.cutoff).to(device)
        for i in range(data.shape[0]):
            cutoff_data[i] = data[i, -self.cutoff:]
        
        # Run stock prices through CNN
        cnn_out = self.cnn(cutoff_data)
        
        # Concat outputs in feature fusion layer 
        feature_in = torch.add(lstm_out, cnn_out)
        
        # Run outputs through feature fusion layer 
        output = self.fusion(feature_in)
        return output

## TreNet Training

In [11]:
"""
Create training loop
"""
def train_loop(n_epochs, X, y, model, loss_fn, optimizer, printout=False, record_loss=False):
    losses = []
    for i in range(n_epochs):
        # Compute prediction and loss 
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print loss per epoch 
        if printout and i % 100 == 0: 
            print(f"Epoch {i}:\n--------------")
            print(f"Train Loss: {np.sqrt(loss.item())}")
            print()
            
        # Record loss per epoch 
        if record_loss: 
            losses.append(np.sqrt(loss.item()))
            
    # Print final loss after training 
    if printout:
        print(f"Final:\n--------------")
        print(f"Train Loss: {np.sqrt(loss.item())}")
        print()
    
    if record_loss:
        return losses

## Test TreNet Model on Trend and Stock Price Data

In [12]:
# Load in trends dataset 
df = pd.read_csv("../data/processed/processed_trends_10.csv")
trends, data = convert_data_points(df)

In [13]:
# Scale data 
trend_scaler = Scaler()
data_scaler = Scaler()

scaled_trends = trend_scaler.fit_transform(trends)
scaled_data = data_scaler.fit_transform(data)

In [14]:
# Extract samples and create train test sets 
trend_X, trend_y = extract_data(scaled_trends, 50, 1)
data_X = scaled_data[51:]

X_train_trend, y_train_trend, X_valid_trend, y_valid_trend, X_test_trend, y_test_trend = train_valid_test_split(trend_X, trend_y)
X_train_data, X_valid_data, X_test_data = train_valid_test_split(data_X)

In [15]:
# Set parameters for model and training 
LSTM_params = {
    'input_dim': 2, 
    'hidden_dim': 32, 
    'num_layers': 1
}

CNN_params = {
    'num_data': 10,
    'layers': 2, 
    'num_filters': [32, 32],
    'dropout': [0.1, 0.1], 
    'conv_size': [2, 4]
}

learning_rate = 0.01
num_epochs = 4000

In [16]:
# Initialize model, loss function, and optimizer
model = TreNet(LSTM_params, CNN_params, 1, 2).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
training_loss = train_loop(num_epochs, [X_train_trend, X_train_data], y_train_trend.reshape(y_train_trend.shape[0], 2), model, loss_fn, optimizer, printout=True, record_loss=True)

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0:
--------------
Train Loss: 0.7039901920442765

Epoch 100:
--------------
Train Loss: 0.12079283411379761

Epoch 200:
--------------
Train Loss: 0.1181159709767101

Epoch 300:
--------------
Train Loss: 0.11708236587736737

Epoch 400:
--------------
Train Loss: 0.11712215509084233

Epoch 500:
--------------
Train Loss: 0.11677338226739689

Epoch 600:
--------------
Train Loss: 0.116406884607723

Epoch 700:
--------------
Train Loss: 0.11638243626863232

Epoch 800:
--------------
Train Loss: 0.11589680215718749

Epoch 900:
--------------
Train Loss: 0.1151638340425978

Epoch 1000:
--------------
Train Loss: 0.11470266291300915

Epoch 1100:
--------------
Train Loss: 0.11301871823677098

Epoch 1200:
--------------
Train Loss: 0.11157638909464349

Epoch 1300:
--------------
Train Loss: 0.11009741947053193

Epoch 1400:
--------------
Train Loss: 0.11183010667050051

Epoch 1500:
--------------
Train Loss: 0.11017417998520528

Epoch 1600:
--------------
Train Loss: 0.1088374329972775

In [18]:
# Compare predictions and actual values 
inverse_test_y = trend_scaler.inverse_transform(y_test_trend.reshape(y_test_trend.shape[0], y_test_trend.shape[2]))

pred_test_y = model([X_test_trend, X_test_data])
inverse_pred_test_y = trend_scaler.inverse_transform(pred_test_y)

In [19]:
print("Test Duration Loss: " + str(loss_fn(inverse_pred_test_y[:, 0], inverse_test_y[:, 0])**(1/2)))
print("Test Slope Loss: " + str(loss_fn(inverse_pred_test_y[:, 1], inverse_test_y[:, 1])**(1/2)))

print("Actual values: \n" + str(inverse_test_y[:10]))

print("Predicted values: \n" + str(inverse_pred_test_y[:10]))

Test Duration Loss: tensor(7.3193)
Test Slope Loss: tensor(20.6998)
Actual values: 
tensor([[ 16.0000,  38.6751],
        [ 16.0000,  15.4913],
        [ 16.0000,  27.3118],
        [ 16.0000,  11.4776],
        [ 16.0000, -17.3519],
        [ 16.0000,   0.2702],
        [ 16.0000,  23.4986],
        [ 22.0000,   7.4000],
        [ 16.0000,  -5.8514],
        [ 16.0000,  14.2896]])
Predicted values: 
tensor([[21.2697,  0.3765],
        [ 4.6700,  0.4781],
        [ 5.6269,  0.4723],
        [12.7729,  0.4285],
        [19.2507,  0.3889],
        [20.0615,  0.3839],
        [ 9.5527,  0.4482],
        [ 3.6829,  0.4842],
        [ 5.2429,  0.4746],
        [16.8651,  0.4035]])


In [20]:
# Save training loss as csv 
pd.DataFrame(training_loss).to_csv("../data/losses/trenet_loss.csv", index=False)